# Meteor Shower Prediction Analysis

This notebook analyzes the performance of our Meteor Shower prediction models (Random Forest and Gradient Boosting), visualizes historical and predicted meteor shower activities, and explores patterns in meteor shower occurrences.

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, r2_score

from src.data_processing.data_loader import DataLoader
from src.models.meteor_shower_predictor import MeteorShowerPredictor
from src.models.gb_meteor_shower_predictor import GBMeteorShowerPredictor
from src.evaluation.prediction_evaluator import evaluate_meteor_shower_predictions

%matplotlib inline
plt.style.use('seaborn')

# List of major meteor showers we're analyzing
SHOWERS = ['Perseids', 'Geminids', 'Leonids', 'Orionids', 'Quadrantids']

## 1. Load and Prepare Data

In [ ]:
loader = DataLoader()
meteor_data = loader.load_meteor_shower_data()

# Split data into training and testing sets
train_data = meteor_data[meteor_data['date'] < '2020-01-01']
test_data = meteor_data[meteor_data['date'] >= '2020-01-01']

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

# Display sample of the data
print("\nSample of meteor shower data:")
print(meteor_data.head())

## 2. Explore Historical Meteor Shower Data

In [ ]:
def plot_shower_intensity(data, shower):
    shower_data = data[data['shower'] == shower]
    plt.figure(figsize=(12, 6))
    plt.scatter(shower_data['date'], shower_data['intensity'], alpha=0.6)
    plt.title(f'Historical Intensity of {shower} Meteor Shower')
    plt.xlabel('Date')
    plt.ylabel('Intensity (meteors per hour)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for shower in SHOWERS:
    plot_shower_intensity(meteor_data, shower)
    
    # Calculate and display shower statistics
    shower_stats = meteor_data[meteor_data['shower'] == shower]['intensity'].describe()
    print(f"\nStatistics for {shower}:")
    print(shower_stats)
    print("\n" + "-"*50)

## 3. Train Meteor Shower Prediction Models

In [ ]:
rf_model = MeteorShowerPredictor()
gb_model = GBMeteorShowerPredictor()

rf_model.train(train_data)
gb_model.train(train_data)

print("Random Forest and Gradient Boosting models trained successfully.")

## 4. Evaluate Model Performance

In [ ]:
def evaluate_models(rf_model, gb_model, test_data):
    rf_predictions = rf_model.predict(test_data)
    gb_predictions = gb_model.predict(test_data)
    
    rf_mse = mean_squared_error(test_data['intensity'], rf_predictions)
    gb_mse = mean_squared_error(test_data['intensity'], gb_predictions)
    
    rf_r2 = r2_score(test_data['intensity'], rf_predictions)
    gb_r2 = r2_score(test_data['intensity'], gb_predictions)
    
    print("Random Forest Model:")
    print(f"Mean Squared Error: {rf_mse:.2f}")
    print(f"R2 Score: {rf_r2:.2f}")
    
    print("\nGradient Boosting Model:")
    print(f"Mean Squared Error: {gb_mse:.2f}")
    print(f"R2 Score: {gb_r2:.2f}")
    
    return rf_predictions, gb_predictions

rf_predictions, gb_predictions = evaluate_models(rf_model, gb_model, test_data)

## 5. Visualize Predictions vs Actual

In [ ]:
def plot_predictions_vs_actual(test_data, rf_predictions, gb_predictions, shower):
    shower_data = test_data[test_data['shower'] == shower]
    shower_rf_pred = rf_predictions[test_data['shower'] == shower]
    shower_gb_pred = gb_predictions[test_data['shower'] == shower]
    
    plt.figure(figsize=(15, 8))
    plt.scatter(shower_data['date'], shower_data['intensity'], alpha=0.6, label='Actual')
    plt.scatter(shower_data['date'], shower_rf_pred, alpha=0.6, label='Random Forest')
    plt.scatter(shower_data['date'], shower_gb_pred, alpha=0.6, label='Gradient Boosting')
    plt.title(f'Actual vs Predicted Intensity for {shower}')
    plt.xlabel('Date')
    plt.ylabel('Intensity (meteors per hour)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for shower in SHOWERS:
    plot_predictions_vs_actual(test_data, rf_predictions, gb_predictions, shower)

## 6. Analyze Prediction Errors

In [ ]:
def analyze_errors(test_data, rf_predictions, gb_predictions):
    rf_errors = test_data['intensity'] - rf_predictions
    gb_errors = test_data['intensity'] - gb_predictions
    
    plt.figure(figsize=(12, 6))
    sns.histplot(rf_errors, kde=True, label='Random Forest')
    sns.histplot(gb_errors, kde=True, label='Gradient Boosting')
    plt.title('Distribution of Prediction Errors')
    plt.xlabel('Error (Actual - Predicted)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
    
    print("Error Statistics:")
    print(f"Random Forest - Mean Error: {rf_errors.mean():.2f}, Std Dev: {rf_errors.std():.2f}")
    print(f"Gradient Boosting - Mean Error: {gb_errors.mean():.2f}, Std Dev: {gb_errors.std():.2f}")

analyze_errors(test_data, rf_predictions, gb_predictions)

## 7. Predict Future Meteor Showers

In [ ]:
def predict_future_showers(rf_model, gb_model, start_date, end_date):
    future_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    future_data = pd.DataFrame({'date': future_dates})
    
    rf_predictions = rf_model.predict(future_data)
    gb_predictions = gb_model.predict(future_data)
    
    future_data['rf_intensity'] = rf_predictions
    future_data['gb_intensity'] = gb_predictions
    
    plt.figure(figsize=(15, 8))
    plt.plot(future_data['date'], future_data['rf_intensity'], label='Random Forest')
    plt.plot(future_data['date'], future_data['gb_intensity'], label='Gradient Boosting')
    plt.title(f'Predicted Meteor Shower Activity ({start_date} to {end_date})')
    plt.xlabel('Date')
    plt.ylabel('Predicted Intensity (meteors per hour)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Find peaks in predictions
    rf_peaks = future_data[future_data['rf_intensity'] > future_data['rf_intensity'].quantile(0.95)]
    gb_peaks = future_data[future_data['gb_intensity'] > future_data['gb_intensity'].quantile(0.95)]
    
    print("Predicted peak meteor shower activities:")
    print("Random Forest Model:")
    print(rf_peaks[['date', 'rf_intensity']].sort_values('rf_intensity', ascending=False).head())
    print("\nGradient Boosting Model:")
    print(gb_peaks[['date', 'gb_intensity']].sort_values('gb_intensity', ascending=False).head())

predict_future_showers(rf_model, gb_model, '2025-01-01', '2025-12-31')

## 8. Analyze Meteor Shower Patterns

In [ ]:
def analyze_shower_patterns(data):
    yearly_peaks = data.groupby(['shower', data['date'].dt.year])['intensity'].max().unstack()
    
    plt.figure(figsize=(15, 8))
    sns.heatmap(yearly_peaks, cmap='YlOrRd', annot=True, fmt='.0f')
    plt.title('Yearly Peak Intensities of Meteor Showers')
    plt.xlabel('Year')
    plt.ylabel('Meteor Shower')
    plt.tight_layout()
    plt.show()
    
    # Analyze periodicity
    for shower in SHOWERS:
        shower_data = data[data['shower'] == shower]
        peak_years = shower_data.groupby(shower_data['date'].dt.year)['intensity'].idxmax()
        peak_intervals = peak_years.diff().dt.days / 365.25  # Convert to years
        
        print(f"\n{shower} - Average years between peaks: {peak_intervals.mean():.2f}")
        print(f"Standard deviation: {peak_intervals.std():.2f}")

analyze_shower_patterns(meteor_data)

print("\nKey observations from the shower pattern analysis:")
print("1. [Your observation about periodicity of different showers]")
print("2. [Your observation about variations in peak intensities over years]")
print("3. [Any other interesting patterns or anomalies you notice]")

## 9. Feature Importance Analysis

In [ ]:
def plot_feature_importance(model, model_name):
    feature_importance = model.feature_importances_
    feature_names = model.feature_names_
    
    importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
    importance_df = importance_df.sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='importance', y='feature', data=importance_df)
    plt.title(f'Feature Importance - {model_name}')
    plt.tight_layout()
    plt.show()

plot_feature_importance(rf_model, 'Random Forest')
plot_feature_importance(gb_model, 'Gradient Boosting')

print("Key insights from feature importance analysis:")
print("1. [Your observation about top features for predicting meteor shower intensity]")
print("2. [Comparison of important features between RF and GB models]")
print("3. [Any surprising or counterintuitive results in feature importance]")

## 10. Model Performance Across Different Showers

In [ ]:
def evaluate_per_shower(test_data, rf_predictions, gb_predictions):
    results = []
    for shower in SHOWERS:
        shower_mask = test_data['shower'] == shower
        shower_actual = test_data.loc[shower_mask, 'intensity']
        shower_rf_pred = rf_predictions[shower_mask]
        shower_gb_pred = gb_predictions[shower_mask]
        
        rf_mse = mean_squared_error(shower_actual, shower_rf_pred)
        gb_mse = mean_squared_error(shower_actual, shower_gb_pred)
        rf_r2 = r2_score(shower_actual, shower_rf_pred)
        gb_r2 = r2_score(shower_actual, shower_gb_pred)
        
        results.append({
            'Shower': shower,
            'RF_MSE': rf_mse,
            'GB_MSE': gb_mse,
            'RF_R2': rf_r2,
            'GB_R2': gb_r2
        })
    
    results_df = pd.DataFrame(results)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Shower', y='value', hue='variable', 
                data=pd.melt(results_df, ['Shower'], var_name='variable', value_name='value'))
    plt.title('Model Performance Across Different Meteor Showers')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print(results_df)
    
    print("\nKey observations on model performance across showers:")
    print("1. [Your observation about which showers are predicted most accurately]")
    print("2. [Comparison of RF and GB performance for different showers]")
    print("3. [Any patterns in prediction difficulty across showers]")

evaluate_per_shower(test_data, rf_predictions, gb_predictions)

## 11. Conclusion and Future Work

In this notebook, we've conducted a comprehensive analysis of meteor shower predictions using both Random Forest and Gradient Boosting models. Key findings include:

1. [Summarize overall model performance, comparing RF and GB]
2. [Discuss any common patterns or interesting differences in meteor shower occurrences]
3. [Highlight any particularly challenging aspects of meteor shower prediction]
4. [Mention any unexpected results or insights gained from the analysis]

Future work could include:
1. Incorporating additional astronomical features to improve prediction accuracy, such as solar activity or interplanetary dust distribution.
2. Developing a hybrid model that combines the strengths of both Random Forest and Gradient Boosting approaches.
3. Extending the model to predict not just intensity, but also the duration and radiant position of meteor showers.
4. Investigating the potential for real-time updating of predictions based on early observed data during a meteor shower event.
5. Exploring the use of deep learning models, particularly for capturing long-term dependencies in meteor shower patterns.
6. Correlating meteor shower predictions with other astronomical phenomena predicted by our system (e.g., planetary positions, solar activity).

This analysis demonstrates the potential of machine learning techniques in predicting complex astronomical phenomena like meteor showers. It provides a foundation for further research into space weather forecasting and our understanding of small body dynamics in the solar system.